# Mediana dla obrazu kolorowego

Idea filtracji medianowej jest dość prosta dla obrazów w odcieniach szarości.
Dla obrazów kolorowych trudniej jest określić kryterium wg. którego szeregowane będą wartości, z których wyznaczana będzie mediana.

Jedną z możliwości wykonania filtracji medianowej dla obrazów kolorowych (na podstawie *The Image Processing Handbook*, J. Russ) jest wykorzystanie następującej definicji mediany:
``mediana to ten piksel z otoczenia, którego odległość do innych pikseli z otoczenia jest najmniejsza''.
Jako miarę odległości wykorzystujemy pierwiastek z sumy kwadratów różnic poszczególnych składowych R,G,B.
Zatem odległość między dwoma pikselami wyraża się wzorem:
\begin{equation}
dRGB = \sqrt{(R_1-R_2)^2+(G_1-G_2)^2+(B_1-B_2)^2}
\end{equation}

Warto zwrócić uwagę, że istnieje wiele możliwości zdefiniowania porównywania wielkości wektorowych (jeden piksel to wektor o trzech składowych).
Można zamiast odległości wykorzystać kąt albo połączyć oba parametry.
Ponadto istnieje możliwość dodania do wektora dodatkowych składowych - tak aby lepiej opisać piksel.

Celem zadania jest implementacja opisanego algorytmu.

1. Wczytaj obraz *lenaRGBSzum.png* (dostępny na git).
2. Zdefiniuj rozmiar okna.
3. Wykonaj pętle po pikselach, dla których okno jest zdefiniowane (pomiń brzeg obrazu).
4. Dla każdego piksela pobierz okno o właściwym rozmiarze.
5. Wykonaj pętle po oknie, wewnątrz której obliczona zostanie suma odległości.
    - Obliczanie różnicy: `window - window[rowWin, colWin]`.
    - Obliczanie kwadratów: `np.square`.
    - Obliczanie pierwiastka: `np.sqrt`.
    - Obliczanie sumy metodą `.sum`.
6. Po obliczeniu macierzy odległości wyznacz argument elementu minimalnego.
Wykorzystaj funkcję `np.argmin`.
Argument funkcji zostanie spłaszczony, jeśli ma więcej niż jeden wymiar.
Aby przekonwertować spłaszczony indeks na indeks macierzy wykorzystaj funkcję `np.unravel_index`.
7. Przypisz odpowiedni wektor wartości do piksela obrazu wynikowego.
8. Wyświetl obraz oryginalny i przefiltrowany.
9. Przeprowadź dwa eksperymenty - dla obrazu _lenaRGB_ oraz _lenaRGBszum_.

In [ ]:
import matplotlib.pyplot as plt
import cv2 as cv
from typing import Tuple
import numpy as np
import os

if not os.path.exists("lenaRGB.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/lenaRGB.png --no-check-certificate
if not os.path.exists("lenaRGBSzum.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/lenaRGBSzum.png --no-check-certificate

Image = np.ndarray
def load_img(path: str, cmap: int = cv.COLOR_BGR2RGB) -> Image:
    img = cv.imread(path)
    return cv.cvtColor(img,cmap)

def display_img(img: Image) -> None:
    plt.imshow(img)
    plt.axis("off")
    plt.show()

In [ ]:

lena = load_img("lenaRGB.png")
lenaNoise = load_img("lenaRGBSzum.png")
display_img(lena)

def filter_median(img: Image, kernel_size: int = 3) -> Image:
    offset = 1
    (height, width, _) = img.shape
    margin = int((kernel_size-1)/2)

    filtered_img = np.empty(img.shape).astype(np.uint8)

    for i in range(margin, width-margin):
        for j in range(margin, height-margin):
            window:np.ndarray = img[i-1:i+1+offset,j-1:j+1+offset,:]
            dgrb = np.empty((kernel_size,kernel_size))
            for k in range(kernel_size):
                for l in range(kernel_size):
                    dgrb[k,l]=np.sqrt(np.sum(np.square(window-window[k,l]),axis=2)).sum()
            (x,y) = np.unravel_index(np.argmin(dgrb),(kernel_size,kernel_size))
            filtered_img[i,j] = window[x, y].astype(np.uint8)
    return filtered_img

filtered_lena = filter_median(lena)
display_img(filtered_lena)
display_img(lenaNoise)
display_img(filter_median(lenaNoise))